# 脉冲神经网络的在线学习


在[关键概念](./concepts-zh.ipynb)一章中，我们介绍了`brainscale`在线学习的基础知识。在本节中，我们将讨论如何基于``brainscale``进行脉冲神经网络（Spiking Neural Networks，SNNs）的在线学习。

基于发放率的循环神经网络（Rate-based RNNs）相比于脉冲神经网络（Spiking Neural Networks，SNNs）更广泛地应用于目前的深度学习任务中。在这种网络中，神经元的输出是连续的浮点数，而非像SNNs中的离散脉冲。``brainscale``提供的``DiagParamDimAlgorithm``算法可以非常高效地用于支持RNNs的在线学习。